In [ ]:
import json
import os
import sys
import math
import datetime
import pathlib
import numpy as np

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
num_rows = 1000 * 1000
num_cols = 20
workload_list = ['core', 'geo', 'classic', 'log', 'ml', 'bi']
scan_exec_pq = f'{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/parquet-scan-columnbatch'

# Predefined workloads, size and scan time

In [ ]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'python3 gen_workloads.py \
        {wl} {num_rows} {num_cols} {base_name}')
    os.system(f'cp -r {base_name} {dir_path} && \
        rm -r {base_name}')
os.chdir(dir_path)
!pwd

In [ ]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_default.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

In [ ]:
os.chdir(f'{PROJ_SRC_DIR}/python')
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'cp general/{base_name}/gen_data/{base_name}.csv data_gen')
    os.system(f'parquet-fromcsv -s general/{base_name}/gen_data/{base_name}_arrow_schema.txt -i \
    general/{base_name}/gen_data/{base_name}.csv -o general/{base_name}.parquet')
    # os.system(f'python3 scripts/parquet_exp.py {base_name} pq_default -r=false -o=false')
    orc_schema = open(f'general/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
    os.system(f'{HOME_DIR}/orc/build/tools/src/csv-import \"{orc_schema}\" \
            general/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
    # os.system(f'mv {base_name}{pq_name_suffix} general/{base_name}.parquet')
    os.system(f'mv {base_name}.orc general/{base_name}.orc')
os.chdir(f'{PROJ_SRC_DIR}/python/general')

In [ ]:
# running experiments
os.system('rm outputs/stats.json')
for i in range(10):
    for wl in workload_list:
        base_name = f'{wl}_r{num_rows}_c{num_cols}'
        os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
        time.sleep(1)
        pq_read = float(os.popen(f'''{scan_exec_pq} \
                --batch_size=1024 ./{base_name}.parquet''').read().split('\n')[0].split(' ')[-2])
        orc_read = float(os.popen(f'''{HOME_DIR}/orc/build/tools/src/orc-scan \
                -b 1024 ./{base_name}.orc''').read().split('\n')[0].split(' ')[-1])
        orc_size = os.path.getsize(f'{base_name}.orc')
        pq_size = os.path.getsize(f'{base_name}.parquet')
        output_stats = {}
        output_stats['workload'] = wl
        output_stats['i'] = i
        output_stats['file'] = 'parquet'
        output_stats['size'] = pq_size
        output_stats['read_time'] = pq_read
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = orc_size
        output_stats['read_time'] = orc_read
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('general_exp', timestamp))

# Predefined workloads, filter (select)

In [ ]:
# generate filters and store in dir
os.chdir(dir_path)
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    filter_dir = f'{wl}_filters'
    os.makedirs(filter_dir, exist_ok=True)
    os.chdir(filter_dir)
    os.system(f'rm -rf point*.csv range*.csv')
    config_name = f'{PROJ_SRC_DIR}/benchmark/generator_v2/filter_config/{wl}_filter.yaml'
    os.system(f'''python3 {PROJ_SRC_DIR}/benchmark/generator_v2/filter_generator.py \
    ../{base_name}/gen_data/{base_name}.csv ../{base_name}/configs/table_config.json \
        {config_name}''')
    os.chdir(dir_path)

In [ ]:
# running experiments
import yaml
import itertools
os.system('rm outputs/stats.json')
output_stats = {}
for wl in workload_list:
    print(f'begin wl: {wl}')
    output_stats['wl'] = wl
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    filter_dir = f'{wl}_filters'
    config_name = f'{PROJ_SRC_DIR}/benchmark/generator_v2/filter_config/{wl}_filter.yaml'
    config = yaml.safe_load(open(config_name))
    config_list = config['filter_config']
    proj_type = 'one'
    for (i, fmt, config) in itertools.product(range(5), ['parquet', 'orc'], config_list):
        output_stats['format'] = fmt
        output_stats['i'] = i
        output_stats['proj_type'] = proj_type
        expectation = config.get('selectivity_expectation')
        output_stats['selectivity'] = expectation
        range_exist = True
        point_exist = True
        try:
            range_filters = pd.read_csv(f'{filter_dir}/range_{expectation}.csv')
        except:
            range_exist = False
        try:
            point_filters = pd.read_csv(f'{filter_dir}/point_{expectation}.csv')
        except:
            point_exist = False
        if range_exist:
            output_stats['filter_type'] = 'range'
            for index, row in range_filters.iterrows():
                val1 = np.int64(float(row['val1'])) if row['dtype'] == 'int' else row['val1']
                val2 = np.int64(float(row['val2'])) if row['dtype'] == 'int' else row['val2']
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                if fmt == 'parquet':
                    cmd = f"parquet-filterscan-noeval -f \
                        {base_name}.parquet -i {row['col_idx']} -l {val1} -r \
                            {val2} -e true"
                    output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                    output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                else:
                    cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                    {base_name}.orc f{row['col_idx']} {row['dtype']} range \
                        {val1} {val2} {proj_type} e"
                    output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                parse_output(output_stats)
        if point_exist:
            output_stats['filter_type'] = 'point'
            for index, row in point_filters.iterrows():
                val1 = np.int64(float(row['value'])) if row['dtype'] == 'int' else row['value']
                val2 = np.int64(float(row['value'])) if row['dtype'] == 'int' else row['value']
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                if fmt == 'parquet':
                    cmd = f"parquet-filterscan-noeval -f \
                        {base_name}.parquet -i {row['col_idx']} -l {val1} -r \
                            {val2} -e true"
                    output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                    output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                else:
                    cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                    {base_name}.orc f{row['col_idx']} {row['dtype']} range \
                        {val1} {val2} {proj_type} e"
                    output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format(f'general_filter', timestamp))